In [ ]:
import os, sys
path_to_add = os.path.dirname(os.path.abspath(os.getcwd()))

if path_to_add not in sys.path:
    sys.path.append(path_to_add)
    
from pathlib import Path
import numpy as np
from cellector.io import create_from_suite2p, create_from_suite3d, create_from_mask_volume, create_from_pixel_data
from cellector.io import propagate_criteria
from cellector.manager import CellectorManager
from cellector.gui import SelectionGUI

In [ ]:
suite2p_dir = Path(r"C:\Path\to\suite2p\results") # this is the one that contains folders for each plane!!!
use_redcell = True # will load the redcell.npy files from each plane folder and use those as a "feature"
clear_existing = False # if True, will delete any existing cellector files in the suite2p folder
autocompute = True # if True, will automatically compute all the features and save them to the cellector files (this takes ~10 seconds depending on your number of ROIs)

# build the roi_processor object
# will load all mask and reference data, and will load saved features from the cellector folder if they exist instead of recomputing them 
# the save directory will be suite2p_dir / cellector
roi_processor = create_from_suite2p(suite2p_dir, use_redcell=use_redcell, clear_existing=clear_existing, autocompute=autocompute)

# if you want to recompute the features, you can either set clear_existing to True (which will remove all existing cellector files...)
# or you can call the compute_features method with use_saved=False like follows
# roi_processor.compute_features(use_saved=False)

In [ ]:
# You can also use the outputs of suite3d, which produces volumetric ROIs
suite3d_dir = Path(r"C:\Path\to\suite3d\results") # this is the one that contains folders for each plane!!!
clear_existing = False # if True, will delete any existing cellector files in the suite2p folder
autocompute = True # if True, will automatically compute all the features and save them to the cellector files (this takes ~10 seconds depending on your number of ROIs)

# build the roi_processor object
# will load all mask and reference data, and will load saved features from the cellector folder if they exist instead of recomputing them 
# the save directory will be suite3d_dir / cellector
roi_processor = create_from_suite3d(suite3d_dir, clear_existing=clear_existing, autocompute=autocompute)

In [ ]:
# Alternative to above, if you have mask data independent from suite2p, use one of the following
save_dir = Path(r"C:\Path\to\save_direcotry") # this is where results will be saved

stats = # List of dictionaries containing mask data (lam, ypix, xpix, for each ROI)
reference_images = # 3D array of reference images - one for each plane
plane_idx = # 1D array relating each mask to the apppropriate reference image
functional_reference_images = None # OPTIONAL! Same shape as reference images but for the functional channel

# if you have stats already, use:
roi_processor = create_from_pixel_data(save_dir, stats, reference_images, plane_idx, functional_references=functional_reference_images)

# if you have a mask volume but not stats, use:
mask_volume = # 3D array, where each slice is an image of the mask for your ROIs
roi_processor = create_from_mask_volume(save_dir, mask_volume, reference_images, plane_idx, functional_references=functional_reference_images)

In [5]:
# Open the GUI
gui = SelectionGUI(roi_processor)

In [ ]:
# If you want to use the criteria selected in the GUI for other sessions, you can propagate them like this:
other_directories = [Path(r"C:\Path\to\other\suite2p"), Path(r"C:\Path\to\another\suite2"), ...] # as many as you like

# First you have to process each folder so the features are computed and stored
# You can do it like this:
for directory in other_directories:
    roi_processor = create_from_suite2p(directory, autocompute=True) # or whichever method you used to create the roi_processor

# Then propagate the criteria from a session you manually curated to all the others like this:
success, failure = propagate_criteria(suite2p_dir, *other_directories)

# Success is a dictionary with keys equal to successful target directories and values set to the names of criteria that were propagated
# Failure is a dictionary with keys equal to failed target directories and values set to the error message for each one

In [ ]:
# Optional: if you want to update the criteria programmatically, you can choose a 
# feature (or multiple) and set new criteria like this:

# Criteria have to be a 2 element list or numpy array, with increasing float values,
# where the first element is the lower bound and the second element is the upper bound
# If you want to ignore the lower / upper bound, set that one to None
feature_with_criteria_to_update = "dot_product"
new_criteria = [None, np.inf] # this will set the criteria to be "dot_product < np.inf" e.g. not use a negative threshold and keep everything below infinity

# Then, if you want to automatically apply the criteria without opening the GUI, you can do this:
for directory in other_directories:
    roi_processor = create_from_suite2p(directory) # or whichever method you used to create the roi_processor
    manager = CellectorManager.make_from_roi_processor(roi_processor)
    
    # you can update criteria programmatically like this
    manager.update_criteria(feature_with_criteria_to_update, new_criteria)
    
    # this will save the updated criteria and idx_selection to cellector directory
    # it will also save empty manual label arrays if they don't exist
    manager.save_all() 

    # Or save everything independently...
    manager.save_criteria()
    manager.save_idx_selected()
    manager.save_manual_selection()
    manager.save_features()


## Handling deprecations
This is a new package, so there is some updates that will impact backwards incompatibility. Some of them will be "obvious" because
they involve changes to the API so things will break or methods won't exist anymore. Sorry. 

Some updates, on the other hand, involve changes in file path convention. This is important to update because if you continue using
cellector with updated versions, your data will be named differently on disk. To address this major annoyance, there are some tools
for fixing old names / data structure quickly. 

In [ ]:
from cellector.io import identify_cellector_folders, update_idx_selection_filenames, update_feature_paths, update_manual_selection_shape

top_dir = "./some/path" # any path that you know contains all the cellector directories you've made
root_dirs = identify_cellector_folders(top_dir) # returns a list of all directories that contain a dir named "cellector" one level down.

update_idx_selection_filenames(root_dirs) # changes targetcell.npy to idx_selection.npy
update_manual_selection_shape(root_dirs) # converts manual_selection to a (2, num_rois) array from an (num_rois, 2) array
update_feature_paths(root_dirs) # uses new feature naming conventions ({fname}_feature.npy) instead of ({fname}.npy) and ({fname}_featurecriteria.npy) instead of ({fname}_criteria.npy)